In [1]:
import os
import random
from matplotlib import axis
import scipy.io as sio
import numpy as np
import math
import mne
import nolds
import joblib
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import KFold, train_test_split
import model as dl  # Ensure this module contains necessary utility functions
import logging
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU
seed = 22
dl.seed_everything(seed)
# EEG data parameters
duration = 2500
low_pass = 0.5
high_pass = 32

In [4]:
def importAndCropData(file_paths, duration, labels, data_type='preprocessed'):
    EEG_list = []
    label = []
    srate = None
    for i, file in enumerate(file_paths):
        try:
            if data_type == 'raw':
                raw = mne.io.read_raw_edf(file, preload=True, encoding='latin1',verbose='Warning')
                raw.filter(l_freq=low_pass, h_freq=high_pass)                # bandpass filter
                raw.notch_filter(50)                            # notch filter
                data = raw.get_data()[0:19]
                channels = raw.info.get('nchan')
                srate = raw.info.get('sfreq')
            else:
                raw = sio.loadmat(file, uint16_codec='latin1')
                data = raw.get('data')[0:19]
                channels = data.shape[0]
                srate = raw.get('srate_function', [None])[0]
                if srate is None:
                    raise ValueError(f"Sampling rate not found in {file}")

            if data.shape[1] > duration:
                epochs = data.shape[1] // duration
                data_crop = data[:,0:epochs*duration]
            else:
                continue
            label += [labels[i]] * epochs
            channels = 19
            data_new = data_crop.reshape(channels, -1, duration).transpose(1, 0, 2)
            EEG_list.append(data_new)

            logging.info(f"Processed file {file}: {epochs} epochs")
        except Exception as e:
            logging.error(f"Error processing file {file}: {e}")
            continue

    if not EEG_list:
        raise ValueError("No data was loaded. Please check the file paths and formats.")
    
    EEG = np.concatenate(EEG_list)
    label = np.array(label)
    logging.info(f"Total epochs: {EEG.shape[0]}, Normal: {np.sum(label == 1)}, "
            f"MCI: {np.sum(label == 0)}")
    return EEG,label,high_pass

In [5]:
import os
import warnings
# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)
# 定义文件夹路径
base_dir = '糖尿病认知障碍与对照脑电数据'
normal_dir = os.path.join(base_dir, '认知正常')
impaired_dir = os.path.join(base_dir, '认知障碍')

# 获取所有的文件路径
normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith('.edf')]
impaired_files = [os.path.join(impaired_dir, f) for f in os.listdir(impaired_dir) if f.endswith('.edf')]

all_files = normal_files + impaired_files
label_single = np.concatenate([np.ones(len(impaired_files)), np.zeros(len(normal_files))],axis=0)
# 将 all_files 和 label_single 中的元素按相同顺序打乱
combined = list(zip(all_files, label_single))
random.shuffle(combined)
all_files[:], label_single[:] = zip(*combined)
original_data,labels,srate = importAndCropData(all_files, duration, label_single, data_type='raw')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband ed

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-08 21:45:53,952 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知障碍/王翠兰.edf: 135 epochs


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband ed

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-08 21:45:54,638 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知障碍/董艳中.edf: 114 epochs


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband ed

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-08 21:46:28,801 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/果春胜.edf: 124 epochs


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband ed

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
2024-11-08 21:46:29,376 - INFO - Processed file 糖尿病认知障碍与对照脑电数据/认知正常/王秀芝.edf: 66 epochs
2024-11-08 21:46:29,969 - INFO - Total epochs: 5643, Normal: 3097, MCI: 2546


In [6]:
train_indices, test_indices = dl.Split_Sets(10, original_data)

# Ensure output directories exist
ensure_dir("EEGData/"+str(srate)+"hz/TrainData")
ensure_dir("EEGData/"+str(srate)+"hz/ValidData")
ensure_dir("EEGData/"+str(srate)+"hz/TestData")

for fold in range(10):
    try:
        # Split into training and test sets
        train_idx = train_indices[fold]
        test_idx = test_indices[fold]

        train_data = original_data[train_idx,:, : ] 
        train_labels = labels[train_idx]
        test_data = original_data[test_idx,:, : ]
        test_labels = labels[test_idx]
        
        # Further split training data into train and validation sets
        train_data_split, valid_data_split, train_labels_split, valid_labels_split = train_test_split(
            train_data, train_labels, test_size=0.1, random_state=seed, stratify=train_labels
        )
        # print(train_data_split.shape,train_labels_split.shape,valid_data_split.shape,valid_labels_split.shape)
        # Convert to PyTorch tensors
        train_tensor = torch.from_numpy(train_data_split).float() # (samples, channels, duration)
        train_labels_tensor = torch.from_numpy(train_labels_split).long()

        valid_tensor = torch.from_numpy(valid_data_split).float()
        valid_labels_tensor = torch.from_numpy(valid_labels_split).long()

        test_tensor = torch.from_numpy(test_data).float()
        test_labels_tensor = torch.from_numpy(test_labels).long()

        # Create TensorDatasets
        train_dataset = TensorDataset(train_tensor, train_labels_tensor)
        valid_dataset = TensorDataset(valid_tensor, valid_labels_tensor)
        test_dataset = TensorDataset(test_tensor, test_labels_tensor)

        # Save datasets
        torch.save(train_dataset, "EEGData/"+str(srate)+f"hz/TrainData/train_data_{fold + 1}_fold_with_seed_{seed}.pth")
        torch.save(valid_dataset, "EEGData/"+str(srate)+f"hz/ValidData/valid_data_{fold + 1}_fold_with_seed_{seed}.pth")
        torch.save(test_dataset, "EEGData/"+str(srate)+f"hz/TestData/test_data_{fold + 1}_fold_with_seed_{seed}.pth")

        logging.info(f"Fold {fold + 1} data saved successfully.")
    except Exception as e:
        logging.error(f"Error processing fold {fold + 1}: {e}")


2024-11-08 21:46:32,388 - INFO - Fold 1 data saved successfully.
2024-11-08 21:46:34,881 - INFO - Fold 2 data saved successfully.
2024-11-08 21:46:37,665 - INFO - Fold 3 data saved successfully.
2024-11-08 21:46:40,180 - INFO - Fold 4 data saved successfully.
2024-11-08 21:46:42,920 - INFO - Fold 5 data saved successfully.
2024-11-08 21:46:45,025 - INFO - Fold 6 data saved successfully.
2024-11-08 21:46:47,151 - INFO - Fold 7 data saved successfully.
2024-11-08 21:46:49,262 - INFO - Fold 8 data saved successfully.
2024-11-08 21:46:51,362 - INFO - Fold 9 data saved successfully.
2024-11-08 21:46:53,463 - INFO - Fold 10 data saved successfully.
